In [ ]:
#library imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import re
import spacy
from collections import Counter
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import LongTensor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filename = '/content/drive/My Drive/Colab/pytorch/pytorch-sign/result.csv'
sldata = pd.read_csv(filename)
print(sldata.shape)
sldata.head()

(3722, 2)


,data,slname
0,183.727366 27.550516 11.044264 9.641405 88.178...,hi
1,209.080695 -1.669812 17.934851 3.740791 71.860...,hi
2,203.873401 35.669062 4.382794 8.047391 80.1403...,hi
3,199.467298 3.350777 14.434275 13.103220 82.467...,hi
4,182.149739 3.310897 10.456150 9.082512 78.2379...,hi


In [ ]:
#calculating data lengths
sldata['data_length'] = sldata['data'].apply(lambda x: len(x.split()))
print(max(sldata['data_length'])) #max_length

7982


In [ ]:
def toarray(x):
    data_list = list(map(float, x.split()))
    arr = np.array(data_list)
    length = len(arr)

    return arr, length

sldata['data_array']= sldata['data'].apply(lambda x: np.array(toarray(x)))
print(sldata['data_array'][0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


[array([183.727366,  27.550516,  11.044264, ...,  60.706998,  55.986877,
       125.916584])
 2444]


In [ ]:
#change slnames to numbering
numbering = {'hi':0, 'meet':1, 'glad':2, 'read':3, 'animal':4, 'long_time':5, 'busy':6, 'end':7, 'yet':8, 'together':9, 'noodle':10, 
             'cold':11, 'different':12, 'state':13, 'no':14, 'medicine':15, 'rest':16, 'please':17, 'first_time':18, 'hobby':19, 'what':20,
             'like':21, 'dog':22, 'nowadays':23, 'eat':24, 'go':25, 'sick':26, 'cough':27, 'disease':28, 'have':29, 'enough':30, 'again':31, 'come':32, 'yes':33, 'two_days_later':34}
sldata['slname_num'] = sldata['slname'].apply(lambda x: numbering[x])
sldata.head()

,data,slname,data_length,data_array,slname_num
0,183.727366 27.550516 11.044264 9.641405 88.178...,hi,2444,"[[183.727366, 27.550516, 11.044264, 9.641405, ...",0
1,209.080695 -1.669812 17.934851 3.740791 71.860...,hi,3328,"[[209.080695, -1.669812, 17.934851, 3.740791, ...",0
2,203.873401 35.669062 4.382794 8.047391 80.1403...,hi,2418,"[[203.873401, 35.669062, 4.382794, 8.047391, 8...",0
3,199.467298 3.350777 14.434275 13.103220 82.467...,hi,2496,"[[199.467298, 3.350777, 14.434275, 13.10322, 8...",0
4,182.149739 3.310897 10.456150 9.082512 78.2379...,hi,1560,"[[182.149739, 3.310897, 10.45615, 9.082512, 78...",0


In [ ]:
#check how balanced the dataset is
Counter(sldata['slname_num'])

Counter({0: 109,
         1: 105,
         2: 104,
         3: 104,
         4: 102,
         5: 109,
         6: 103,
         7: 103,
         8: 103,
         9: 103,
         10: 111,
         11: 109,
         12: 109,
         13: 106,
         14: 108,
         15: 111,
         16: 106,
         17: 105,
         18: 106,
         19: 107,
         20: 108,
         21: 110,
         22: 105,
         23: 106,
         24: 104,
         25: 105,
         26: 106,
         27: 104,
         28: 112,
         29: 106,
         30: 107,
         31: 105,
         32: 109,
         33: 104,
         34: 108})

In [ ]:
X = list(sldata['data_array'])
y = list(sldata['slname_num'])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
class SLDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx][0], self.y[idx], self.X[idx][1] # sequence(tensor로), label, length

In [ ]:
train_ds = SLDataset(X_train, y_train)
test_ds = SLDataset(X_test, y_test)
#print(train_ds.__getitem__(0))

In [ ]:
# Hyperparameters
batch_size = 100
input_size = 26
sequence_length = 307 #7982/26
hidden_size = 256
num_layers = 2
num_classes = 35
learning_rate=0.001


def sort_batch(seqs, targets, lengths):
    """
    Sort a minibatch by the length of the sequences with the longest sequences first
    return the sorted batch targets and sequence lengths.
    This way the output can be used by pack_padded_sequences(...)
    """
    seq_lengths, perm_idx = lengths.sort(0, descending=True)
    seq_tensor = seqs[perm_idx]
    target_tensor = targets[perm_idx]
    return seq_tensor, target_tensor, seq_lengths

def pad_and_sort_batch(data):
    """
    data: is a list of tuples with (seqs, targs, lengths)
             where 'seqs' is a tensor of arbitrary shape
             and targs/lengths are scalars
    Returns a padded tensor of sequences sorted from longest to shortest, 
    """
    batch_size = len(data)
    batch_split = list(zip(*data))
    seqs, targs, lengths = batch_split[0], batch_split[1], batch_split[2]
    max_length = 7982
    padded_seqs = np.zeros((batch_size, max_length))
    for i, l in enumerate(lengths):
        padded_seqs[i, 0:l] = seqs[i][0:l]

    return sort_batch(torch.from_numpy(padded_seqs.astype(np.float32)), torch.tensor(targs), torch.tensor(lengths))

# Load Data
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=pad_and_sort_batch)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True, collate_fn=pad_and_sort_batch)

#for x, y, l in train_dl:
    #print(x.shape)
    #print(y.shape)
    #print(l.shape)

In [ ]:
def train_model(model, num_epochs, learning_rate):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=learning_rate) #optimizer
    for epoch in range(num_epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.view(x.shape[0], -1, 26)
            y_pred = model(x, l)
            loss = F.cross_entropy(y_pred, y) #loss function

            # backward
            optimizer.zero_grad()
            loss.backward()

            #optimize
            optimizer.step()

            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = validation_metrics(model, test_dl)
        if epoch % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f" % (sum_loss/total, val_loss, val_acc*100))

def validation_metrics (model, test_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    #batch로 들어가는지 1 instance로 들어가는지 알기, batch로 들어가면 밑에 float().sum()필요함
    for x, y, l in test_dl:
        x=x.view(x.shape[0], -1, 26)
        y_hat = model(x, l)
        #loss 제대로 나오는지 확인 (문서)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        # float랑 sum 삭제
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    print(f'Got {correct} / {total} with accuracy {float(correct)/float(total)*100:.2f}')
    return sum_loss/total, correct/total

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        #self.fc = nn.Linear(hidden_size, num_classes)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
    
    def forward(self, x, l):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        #x_pack = pack_padded_sequence(x, l, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x, (h0, c0))
        out_pack = out_pack.reshape(out_pack.shape[0], -1)
        out = self.fc(out_pack)
        #out = self.fc(ht[-1])
        return out

In [ ]:
# Initialize network
model = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes)

In [ ]:
train_model(model, 20, 0.001) # model, num_epoch, learning_rate

Got 671.0 / 745 with accuracy 90.07
Got 688.0 / 745 with accuracy 92.35
train loss 0.231, val loss 0.279, val accuracy 92.349
Got 721.0 / 745 with accuracy 96.78
Got 725.0 / 745 with accuracy 97.32
Got 721.0 / 745 with accuracy 96.78
Got 718.0 / 745 with accuracy 96.38
